# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
from utils import *
# Set your preferred name
USER = "Jianrong"
# Set the data and output directories
DATA_ROOT = '/home/yxcheng'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Fibromyalgia'
OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = False

In [139]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb']

If no match is found, jump directly to GEO in Part 2.2

In [3]:
trait_subdir = "TCGA_Breast_Cancer_(BRCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_BRCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [29]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [30]:
clinical_data_cols[:10]

['AJCC_Stage_nature2012',
 'Age_at_Initial_Pathologic_Diagnosis_nature2012',
 'CN_Clusters_nature2012',
 'Converted_Stage_nature2012',
 'Days_to_Date_of_Last_Contact_nature2012',
 'Days_to_date_of_Death_nature2012',
 'ER_Status_nature2012',
 'Gender_nature2012',
 'HER2_Final_Status_nature2012',
 'Integrated_Clusters_no_exp__nature2012']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

In [ ]:
candidate_age_cols = ['Age_at_Initial_Pathologic_Diagnosis_nature2012', 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['Gender_nature2012', 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [5]:
dataset = 'GEO'
trait_subdir = "Fibromyalgia"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE67311']

Repeat the below steps for all the accession numbers

In [6]:
cohort = accession_num = "GSE67311"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/home/yxcheng/GEO/Fibromyalgia/GSE67311/GSE67311_family.soft.gz',
 '/home/yxcheng/GEO/Fibromyalgia/GSE67311/GSE67311_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [7]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Peripheral Blood Gene Expression in Fibromyalgia Patients Reveals  Potential Biological Markers and Physiological Pathways"
!Series_summary	"Fibromyalgia (FM) is a common pain disorder characterized by dysregulation in the processing of pain. Although FM has similarities with other rheumatologic pain disorders, the search for objective markers has not been successful. In the current study we analyzed gene expression in the whole blood of 70 fibromyalgia patients and 70 healthy matched controls. Global molecular profiling revealed an upregulation of several inflammatory molecules in FM patients and downregulation of specific pathways related to hypersensitivity and allergy. There was a differential expression of genes in known pathways for pain processing, such as glutamine/glutamate signaling and axonal development. We also identified a panel of candidate gene expression-based classifiers that could establish an objective blood-based molecular diagnostic to objectively i

In [8]:
clinical_data.head()

,!Sample_geo_accession,GSM1644447,GSM1644448,GSM1644449,GSM1644450,GSM1644451,GSM1644452,GSM1644453,GSM1644454,GSM1644455,...,GSM1644579,GSM1644580,GSM1644581,GSM1644582,GSM1644583,GSM1644584,GSM1644585,GSM1644586,GSM1644587,GSM1644588
0,!Sample_characteristics_ch1,diagnosis: healthy control,diagnosis: healthy control,diagnosis: healthy control,diagnosis: healthy control,diagnosis: healthy control,diagnosis: healthy control,diagnosis: healthy control,diagnosis: healthy control,diagnosis: healthy control,...,diagnosis: fibromyalgia,diagnosis: fibromyalgia,diagnosis: fibromyalgia,diagnosis: fibromyalgia,diagnosis: fibromyalgia,diagnosis: fibromyalgia,diagnosis: fibromyalgia,diagnosis: fibromyalgia,diagnosis: fibromyalgia,diagnosis: fibromyalgia
1,!Sample_characteristics_ch1,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,...,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood
2,!Sample_characteristics_ch1,fiqr score: 8.5,fiqr score: -2.0,fiqr score: 9.8,fiqr score: 0.5,fiqr score: -1.0,fiqr score: -0.5,fiqr score: 2.2,fiqr score: -2.0,fiqr score: -2.0,...,fiqr score: 41.8,fiqr score: 54.5,fiqr score: 63.0,fiqr score: 64.0,fiqr score: 67.2,fiqr score: 17.8,fiqr score: 17.8,fiqr score: 41.7,fiqr score: 81.2,fiqr score: 54.7
3,!Sample_characteristics_ch1,bmi: 36,bmi: 34,bmi: 33,bmi: 22,bmi: 24,bmi: 28,bmi: 23,bmi: 48,bmi: 48,...,bmi: 22,bmi: 25,bmi: 29,bmi: 38,bmi: 28,bmi: 37,bmi: 0,bmi: 38,bmi: 20,bmi: 34
4,!Sample_characteristics_ch1,migraine: No,migraine: No,migraine: No,migraine: No,migraine: No,migraine: No,migraine: No,migraine: No,migraine: No,...,migraine: No,migraine: Yes,migraine: Yes,migraine: No,migraine: Yes,migraine: Yes,migraine: -,migraine: Yes,migraine: Yes,migraine: No


In [9]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['diagnosis: healthy control', 'diagnosis: fibromyalgia'],
 1: ['tissue: peripheral blood'],
 2: ['fiqr score: 8.5',
  'fiqr score: -2.0',
  'fiqr score: 9.8',
  'fiqr score: 0.5',
  'fiqr score: -1.0',
  'fiqr score: -0.5',
  'fiqr score: 2.2',
  'fiqr score: 15.3',
  'fiqr score: 4.0',
  'fiqr score: 29.3',
  'fiqr score: 27.2',
  'fiqr score: 5.0',
  'fiqr score: 1.0',
  'fiqr score: 2.5',
  'fiqr score: 3.0',
  'fiqr score: -1.5',
  'fiqr score: 1.3',
  'fiqr score: 21.7',
  'fiqr score: -1.2',
  'fiqr score: 4.3',
  'fiqr score: 6.5',
  'fiqr score: 2.0',
  'fiqr score: 11.7',
  'fiqr score: 15.0',
  'fiqr score: 6.0',
  'fiqr score: 14.2',
  'fiqr score: -0.2',
  'fiqr score: 12.8',
  'fiqr score: 15.7',
  'fiqr score: 0.0'],
 3: ['bmi: 36',
  'bmi: 34',
  'bmi: 33',
  'bmi: 22',
  'bmi: 24',
  'bmi: 28',
  'bmi: 23',
  'bmi: 48',
  'bmi: 25',
  'bmi: 46',
  'bmi: 32',
  'bmi: 31',
  'bmi: 21',
  'bmi: 27',
  'bmi: 39',
  'bmi: 52',
  'bmi: 37',
  'bmi: 0',
  'bmi: 38',
  'bm

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [10]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Fibromyalgia\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Fibromyalgia\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [11]:
is_gene_availabe = True
trait_row = 1
age_row = 2
gender_row = 3

trait_type = 'binary'

In [12]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [13]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'tissue: Brain':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    
    """
    if age_string.startswith('age: '): 
        try:
            
            age_value = float(age_string.split('age: ')[1].rstrip(' weeks'))
            return age_value
        except ValueError:
            return None 
    else:
        return None  


def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: female':
        return 0
    elif gender_string == 'gender: male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Heart'
example_age = 'age: 0.2 weeks'
example_gender = 'gender: male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, 0.2, 1)

In [150]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM4064970,GSM4064971,GSM4064972,GSM4064973,GSM4064974,GSM4064975,GSM4064976,GSM4064977,GSM4064978,GSM4064979,...,GSM4065020,GSM4065021,GSM4065022,GSM4065023,GSM4065024,GSM4065025,GSM4065026,GSM4065027,GSM4065028,GSM4065029
Brugada Syndrome,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Age,24.0,112.0,8.0,24.0,0.6,72.0,24.0,36.0,52.0,20.0,...,60.0,52.0,0.0,0.0,52.0,0.0,0.0,2.0,2.0,144.0
Gender,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


### Genetic data preprocessing and final filtering

In [151]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM4064970,GSM4064971,GSM4064972,GSM4064973,GSM4064974,GSM4064975,GSM4064976,GSM4064977,GSM4064978,GSM4064979,...,GSM4065020,GSM4065021,GSM4065022,GSM4065023,GSM4065024,GSM4065025,GSM4065026,GSM4065027,GSM4065028,GSM4065029
ID,,,,,,,,,,,,,,,,,,,,,
ILMN_1343291,9204.301266,5075.171525,10051.290505,12123.697693,10574.956805,12959.006463,4846.616019,15194.394130,4235.706279,6740.578348,...,15830.340787,15243.439606,14769.689168,13909.008710,10192.845233,14354.926566,16041.994931,13428.226366,1946.162525,11239.665798
ILMN_1651209,4.004890,5.342496,6.945065,8.094816,7.528906,4.823480,4.631520,9.696207,9.905800,8.640280,...,11.449560,24.699160,19.547560,40.412700,14.336090,22.246680,55.300770,112.238000,117.986800,16.114540
ILMN_1651228,153.984000,3792.218000,710.980200,5711.639000,114.615400,934.037900,20490.110000,8772.020000,934.037900,620.736600,...,99.317010,4045.860000,288.874000,490.598300,1090.540000,531.987500,1802.528000,956.679300,11.410130,637.642700
ILMN_1651229,4758.649323,4885.010970,6980.379696,3278.166966,4653.547573,4973.441489,591.788287,41.117053,1813.969520,4783.730491,...,5714.604457,5482.568242,5060.292222,6118.734244,7730.961740,6244.711505,5705.128276,7137.665780,8704.834259,7614.901015
ILMN_1651235,204.614435,51.636499,7.295319,6.614813,310.366424,1395.234969,236.749618,128.245911,5.101588,524.190165,...,1187.466338,714.294061,1679.476338,1384.370474,1021.047075,1435.140896,591.893404,2622.778277,376.727496,1150.683846


In [152]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['ILMN_1343291',
 'ILMN_1651209',
 'ILMN_1651228',
 'ILMN_1651229',
 'ILMN_1651235',
 'ILMN_1651236',
 'ILMN_1651237',
 'ILMN_1651238',
 'ILMN_1651254',
 'ILMN_1651260',
 'ILMN_1651262',
 'ILMN_1651268',
 'ILMN_1651278',
 'ILMN_1651282',
 'ILMN_1651285',
 'ILMN_1651286',
 'ILMN_1651292',
 'ILMN_1651303',
 'ILMN_1651309',
 'ILMN_1651315']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [153]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286', 'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315']\n"

If not required, jump directly to the gene normalization step

In [154]:
requires_gene_mapping = True

In [155]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Source_Reference_ID': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': ['DQ516750', 'DQ883654', 'DQ668364', 'DQ516785', 'DQ854995'], 'Symbol': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Protein_Product': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5270161.0, 426059

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [156]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Source_Reference_ID': ['ERCC-00162', 'ER

In [157]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [158]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

12 input query terms found dup hits:	[('ABCC13', 2), ('ABCC6P1', 2), ('ABCC6P2', 3), ('ADAM6', 3), ('AGAP11', 2), ('ALOX12P2', 2), ('ANXA
91 input query terms found no hit:	['7A5', 'A26C3', 'A2BP1', 'A2LD1', 'AADACL1', 'AARS', 'ABCA11', 'ABHD7', 'ABHD9', 'ABP1', 'ACCN1', '
4 input query terms found dup hits:	[('ATP6AP1L', 2), ('ATXN8OS', 2), ('BAGE2', 2), ('BRD7P3', 2)]
395 input query terms found no hit:	['ARS2', 'ARSE', 'ARVP6125', 'ASAM', 'ASAP1IT1', 'ASFMR1', 'ASNA1', 'ATHL1', 'ATP5A1', 'ATP5B', 'ATP
2 input query terms found dup hits:	[('C2orf83', 2), ('C5orf60', 2)]
638 input query terms found no hit:	['C17orf72', 'C17orf73', 'C17orf74', 'C17orf76', 'C17orf77', 'C17orf79', 'C17orf80', 'C17orf81', 'C1
4 input query terms found dup hits:	[('CCDC144NL', 2), ('CCT6P1', 2), ('CDR1', 2), ('CLEC4GP1', 2)]
113 input query terms found no hit:	['CCBL1', 'CCBL2', 'CCBP2', 'CCDC101', 'CCDC104', 'CCDC105', 'CCDC108', 'CCDC109A', 'CCDC109B', 'CCD
8 input query terms found dup hits:	[('COPG2IT1

In [159]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [160]:
print(f"The merged dataset contains {len(merged_data)} samples.")
merged_data

The merged dataset contains 60 samples.


,Brugada Syndrome,Age,Gender,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
GSM4064970,0.0,24.0,1.0,62.355348,14.440420,2460.196694,4.686300,6.105481,587.097412,16.382563,...,475.836640,10.585559,1858.563000,309.923000,238.824645,16.280420,27.983100,925.877535,457.178400,1711.636700
GSM4064971,0.0,112.0,1.0,10.278570,7.569081,7293.759251,5.559912,4.979804,327.278673,6.757537,...,114.519993,10.859937,452.635850,432.893700,342.556525,6.001372,328.690900,323.831265,1071.818000,1576.779250
GSM4064972,0.0,8.0,0.0,54.362789,5.384082,1536.235125,195.513335,65.989054,2739.802065,109.323537,...,907.482433,25.361408,2418.226000,6.477754,6.872454,10.014170,182.508200,1914.635812,6.965921,1755.359300
GSM4064973,0.0,24.0,1.0,5.764988,8.260571,4517.352275,5.725376,6.391028,806.762413,45.303023,...,492.995270,13.643631,568.066500,3557.904000,933.398150,7.851095,308.473100,1901.632360,165.038800,1326.673000
GSM4064974,0.0,0.6,0.0,23.992323,18.493197,4215.080680,17.109983,7.504535,923.792662,9.095975,...,449.435800,13.533547,1767.799450,235.760200,122.383625,45.240360,30.014700,901.722045,352.542300,1327.331950
GSM4064975,0.0,72.0,1.0,31.366236,5.428562,1180.786270,5.852868,22.394404,958.550157,73.304113,...,901.280519,16.117974,2119.380000,762.076200,391.236405,9.205301,4.563783,2633.665940,5.055491,926.780490
GSM4064976,0.0,24.0,0.0,267.552914,8.712279,8764.367836,37.777003,438.316512,4.904852,6.736836,...,231.004713,11.046941,401.668940,7046.628000,244.577066,3.651659,2042.331000,1020.103037,162.418400,8.467687
GSM4064977,0.0,36.0,1.0,1879.845723,287.377749,7.111364,5.390598,245.963630,80.346495,1189.334982,...,320.578720,10.155646,733.606260,7.825502,93.016329,219.472800,7.363958,8311.804700,4.340990,169.654780
GSM4064978,0.0,52.0,1.0,112.182100,7.417203,3260.866611,7.106071,10.197472,268.349773,17.072828,...,369.459033,8.997515,181.475945,11267.170000,1351.171000,7.917643,126.146200,3719.357300,241.948900,887.739150
GSM4064979,0.0,20.0,1.0,78.592842,30.706918,1559.723056,9.450259,10.040594,611.063243,74.828051,...,644.490500,11.755870,2287.628500,26.475930,189.163285,12.040690,18.647120,1397.042660,4.680267,1564.791560


In [161]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Brugada Syndrome', the least common label is '1.0' with 19 occurrences. This represents 31.67% of the dataset.
The distribution of the feature 'Brugada Syndrome' in this dataset is fine.

Quartiles for 'Age':
  25%: 0.375
  50% (Median): 24.0
  75%: 52.0
Min: 0.0
Max: 144.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 30 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [162]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

A new JSON file was created at: ../../output1/Yuxuan/Brugada-Syndrome/cohort_info.json


In [163]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)
#merged_data

### 3. Do regression & Cross Validation

In [164]:
from utils import *

In [165]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE136992,True,True,False,True,True,60,


In [166]:
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'GSE136992'

In [167]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,Brugada Syndrome,Age,Gender,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.0,24.0,1.0,62.355348,14.440420,2460.196694,4.686300,6.105481,587.097412,16.382563,...,475.836640,10.585558,1858.56300,309.923000,238.824645,16.280420,27.9831,925.877535,457.178400,1711.63670
1,0.0,112.0,1.0,10.278570,7.569081,7293.759251,5.559912,4.979804,327.278673,6.757537,...,114.519993,10.859937,452.63585,432.893700,342.556525,6.001372,328.6909,323.831265,1071.818000,1576.77925
2,0.0,8.0,0.0,54.362789,5.384082,1536.235125,195.513335,65.989054,2739.802065,109.323537,...,907.482433,25.361408,2418.22600,6.477754,6.872454,10.014170,182.5082,1914.635812,6.965921,1755.35930
3,0.0,24.0,1.0,5.764988,8.260571,4517.352275,5.725376,6.391028,806.762413,45.303023,...,492.995270,13.643631,568.06650,3557.904000,933.398150,7.851095,308.4731,1901.632360,165.038800,1326.67300
4,0.0,0.6,0.0,23.992323,18.493197,4215.080680,17.109983,7.504535,923.792662,9.095975,...,449.435800,13.533547,1767.79945,235.760200,122.383625,45.240360,30.0147,901.722045,352.542300,1327.33195


In [168]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [169]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [170]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [171]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [172]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 56.67% ± 20.68%


In [175]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 95.00% ± 6.67%


In [176]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 4. Discussion and report

In [177]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0429
Found 50 genes with non-zero coefficients associated with the trait 'Brugada Syndrome' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
Variable  Coefficient  Absolute Coefficient
   SFXN5     6.968654              6.968654
    HNMT     5.931100              5.931100
   LIMK2    -5.185019              5.185019
  PDLIM2     5.138317              5.138317
   EXOC3    -4.691808              4.691808
  KCNK15    -4.399284              4.399284
    UGP2     3.874205              3.874205
    LCAT     3.625004              3.625004
 RHOBTB1    -3.490019              3.490019
   METRN     3.480335              3.480335
   PKHD1     3.149871              3.149871
    CHDH     3.043823              3.043823
    KPRP    -3.011505              3.011505
 TNFSF12     2.901458              2.901458
 MADCAM1    -2.832544              2.832544
    CES2     2.74821

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [178]:

interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0429
Found 247 significant genes associated with the trait 'Brugada Syndrome' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient      p_value  corrected_p_value
     BSN     0.330125 5.704162e-13       4.845115e-09
    DPP6     0.340749 4.314206e-13       4.845115e-09
     MT3     0.279684 1.196420e-12       6.774928e-09
 PPP2R2C     0.348435 1.795095e-11       7.623769e-08
   GAP43     0.341790 5.879802e-11       1.664768e-07
    SCG3     0.288491 5.537623e-11       1.664768e-07
   KIF5A     0.294121 7.789659e-11       1.890439e-07
  SNAP25     0.273705 1.313348e-10       2.788894e-07
    MAP2     0.305473 3.887751e-10       7.338346e-07
   GNAO1     0.277382 5.207726e-10       8.846884e-07
    NNAT     0.310292 5.988944e-10       9.249108e-07
    CHD5     0.285377 1.723972e-09       2.440569e-06
    RTN1     0.316324 2.505213e-09       2.862876e-06
   TRIM9     0.250064 2.

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
